# Annotation Day!

In [1]:
import dill
from tqdm import tqdm_notebook
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, DataRange1d
from bokeh.layouts import layout

Since there are just shy of 300 FDA approvals (times 120 days of closing prices and sales volumes), I'm going to attempt to compare the closing price graphs over 120 day periods before and after each FDA announcement and annotate the trials as a success or failure by eye. My exploratory plots had several plainly visible de-listings and price collapses that showed through the noise of a 300-line plot, so I belive plotting one by one will show the information needed. 

I doubt this is 100% kosher, but I think it'll get the job done and help me get back on schedule. I have a pretty good idea how pharma stocks behave and it should be pretty quick to classify my training data along binary lines. 

In [2]:
data = dill.load(open("stock_price_training_slices.pkl", "r"))

In [3]:
output_notebook()

Loading BokehJS ...

Recall the structure of my training data:

```data[x] = ("Ticker", 
            pdufaDate, 
          ([Close Prices in preceding 120 Days], [Volume in preceding 120 Days]), 
          ([Close Prices in following 120 Days], [Volume in following 120 Days])
          )```

Much like in exploratory plots, I'm going to plot the 

In [4]:
datum = data[0]

In [5]:
def score_plots(event): #show two plots for annotating
    company = event[0]
    date = event[1]
    p_closes = event[2][0]
    p_volumes = event[2][1]
    f_closes = event[3][0]
    f_volumes = event[3][1]
    
    #generate plot of preceding close prices
    p_close_plot = figure(
        title = "120 Days Preceding "+company+" "+date+" Close Prices",
        plot_width=350,
        plot_height=350
    )
    p_close_plot.line([x for x in range(len(p_closes), 0, -1)], p_closes)
    p_close_plot.x_range = DataRange1d(flipped = True)
    p_close_plot.xaxis.axis_label = 'Days prior to PDUFA'
    
    #generate plot of following close prices
    f_close_plot = figure(
        title = "120 Days Following "+company+" "+date+" Close Prices",
        plot_width=350,
        plot_height=350
    )
    f_close_plot.line([x for x in range(len(f_closes))], f_closes)
    f_close_plot.xaxis.axis_label = 'Days Following to PDUFA'
    
    #layout and show the plots
    l = layout([p_close_plot, f_close_plot],[])
    show(l)
    

Here are some example plots of the 120 day windows before and after an FDA announcement. I'll have to hand annotate these tonight. 

In [6]:
for datum in tqdm_notebook(data[:5]):
    score_plots(datum)